In [ ]:
"""
This notebook creates a Bayesian geospatial gravity model to estimate the bike ride demand between
stations based on previous data. It also goes over visualizing the results with a flow map.
"""

import glob
import os

import arviz as az
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import pymc as pm
import seaborn as sns

from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import haversine_distances

# File name for the final model results
MODEL_RESULTS_FILE = "../models/time_series.nc"

# File path to the saved station hierarchy model
HIERARCHICAL_MODEL_FILE_PATH = "../models/station_pop_model_results.nc"

In [23]:
# Load in the data
df_bike = pd.read_pickle("../data/indego_bike_data.pkl")

# Subset to the last two years of data
df_bike["start_time"] = pd.to_datetime(
    df_bike["start_time"], format="mixed"
)
cutoff_date = pd.to_datetime("2025-08-16") - pd.DateOffset(years=2)
df_bike_active = df_bike[df_bike["start_time"] >= cutoff_date].copy()

print(df_bike.head())

       trip_id  duration          start_time             end_time  \
0  144361832.0      12.0 2017-07-01 00:04:00  2017-07-01 00:16:00   
1  144361829.0      31.0 2017-07-01 00:06:00  2017-07-01 00:37:00   
2  144361830.0      15.0 2017-07-01 00:06:00  2017-07-01 00:21:00   
3  144361831.0      15.0 2017-07-01 00:06:00  2017-07-01 00:21:00   
4  144361828.0      30.0 2017-07-01 00:07:00  2017-07-01 00:37:00   

   start_station  start_lat  start_lon  end_station    end_lat    end_lon  \
0         3160.0  39.956619 -75.198624       3163.0  39.949741 -75.180969   
1         3046.0  39.950119 -75.144722       3101.0  39.942951 -75.159554   
2         3006.0  39.952202  -75.20311       3101.0  39.942951 -75.159554   
3         3006.0  39.952202  -75.20311       3101.0  39.942951 -75.159554   
4         3046.0  39.950119 -75.144722       3101.0  39.942951 -75.159554   

  bike_id  plan_duration trip_route_category passholder_type bike_type  
0   11883           30.0             One Way     

In [72]:
# Load station data, including whether or not each station is active
df_stations = pd.read_csv("../data/indego-stations-2025-07-01.csv")
df_stations["Station_ID"] = df_stations["Station_ID"].astype("Int64")

# Keep only station IDs where status is "Active"
active_ids = df_stations.loc[df_stations["Status"] == "Active", "Station_ID"]
print(f"Number of active stations: {len(active_ids)}")
print(f"Active station IDs: {active_ids.tolist()}")

# Keep rides where BOTH start and end stations are active
df_bike_active = df_bike[
    df_bike["start_station"].isin(active_ids) & df_bike["end_station"].isin(active_ids)
]

# Setting some basic variables for downstream calculation
n_stations = len(active_ids)
station_lookup = {id: i for i, id in enumerate(active_ids)}
station_ids_in_subset = sorted(pd.concat([df_bike_active['start_station'], df_bike_active['end_station']]).unique())
station_lookup = {id: i for i, id in enumerate(station_ids_in_subset)}
id_to_idx_map = {v: k for k, v in station_lookup.items()}

print(df_bike_active.head())

Number of active stations: 272
Active station IDs: [3000, 3005, 3006, 3007, 3008, 3009, 3010, 3012, 3014, 3015, 3016, 3017, 3018, 3019, 3020, 3021, 3022, 3024, 3025, 3026, 3028, 3029, 3030, 3031, 3032, 3033, 3034, 3035, 3037, 3038, 3039, 3040, 3041, 3047, 3049, 3051, 3052, 3053, 3054, 3055, 3056, 3057, 3058, 3059, 3060, 3061, 3062, 3063, 3064, 3065, 3066, 3067, 3068, 3069, 3072, 3073, 3074, 3075, 3077, 3078, 3086, 3088, 3093, 3096, 3097, 3098, 3099, 3100, 3101, 3102, 3104, 3106, 3107, 3110, 3111, 3112, 3113, 3114, 3115, 3116, 3117, 3118, 3119, 3120, 3123, 3124, 3125, 3150, 3152, 3154, 3155, 3156, 3157, 3158, 3159, 3160, 3161, 3162, 3163, 3164, 3165, 3166, 3168, 3169, 3181, 3182, 3183, 3184, 3185, 3187, 3188, 3196, 3197, 3200, 3201, 3203, 3204, 3205, 3207, 3208, 3209, 3210, 3211, 3212, 3213, 3226, 3235, 3236, 3237, 3238, 3240, 3241, 3243, 3245, 3246, 3247, 3248, 3249, 3250, 3251, 3252, 3253, 3254, 3255, 3256, 3258, 3259, 3261, 3263, 3264, 3265, 3266, 3267, 3271, 3272, 3275, 3276, 3277, 

The final model for this project is a Bayesian geospatial gravity model. In simple terms, this model
can treat each bike station as a planet in outer space, where their mass (popularity) attracts trips
and the distance between each other creates friction that reduces the number of trips.

The model can predict the number of trips between any two stations (but it cannot predict the 
number of trips between the same station) by considering the origin popularity, the destination
popularity, and the distance. Combining each factor into a formula can get the expected number of 
trips for each origin-destination pair. This can help quantify the "friction of distance" and 
how much that reduces the likelihood of a trip.

Note that this is diferent from the hierarchical model. That predicted the origin popularity for each 
day of the week, while this model can quantify the popularity between stations.

This is also different from the static transition metric, which just showed historical trends, while
this model aims to predict the demand.

To do this, I first need to load in the hiararchical model to get the popularity scores for each
station ID.

In [ ]:
idata_station = az.from_netcdf(HIERARCHICAL_MODEL_FILE_PATH)
posterior_samples = az.extract(idata_station)
station_pop_est = posterior_samples['station_log_popularity'].mean(dim='sample').values

# Create the station popularity score
station_popularity = pd.Series(np.exp(station_pop_est), index=station_lookup.values())

# Map each station ID to the station popularity
station_id_to_pop_map = {st_id: station_popularity[idx] for idx, st_id in id_to_idx_map.items()}

Next, I need to create the Origin-Destination Matrix. This is a fundamental tool for general
transportation analysis to help visualize the popularity scores.

In [41]:
od_matrix = pd.crosstab(df_bike_active['start_station'], df_bike_active['end_station'])

print(od_matrix)

end_station    3000.0  3005.0  3006.0  3007.0  3008.0  3009.0  3010.0  3012.0  \
start_station                                                                   
3000.0            269       3       4       7       4       0       5       5   
3005.0            363    2289      64     506      32     148     890     176   
3006.0            651      48    3880     171       3    1021     431     524   
3007.0           1166     636     167    5955     201     271    4217     995   
3008.0            304      17       9     166    1355      44      86       9   
...               ...     ...     ...     ...     ...     ...     ...     ...   
3418.0              6       0       0       0       1       0       0       0   
3421.0              3       0       0      10       0       0       0       0   
3422.0              6       0       1       0       0       7       3       0   
3425.0              6       0       0       0       3       0       0       0   
3430.0              2       

Last major component I need before building my model is a distance dataframe, that can calculate
the distance between stations. I do this by first making a new dataframe with the lat/lon coords for
each station ID, and then calculating the Euclidean distances by convering the lat/long coords
to radians.

In [ ]:
df_stations = df_bike_active[['start_station', 'start_lat', 'start_lon']].rename(
    columns={'start_station': 'station_id', 'start_lat': 'latitude', 'start_lon': 'longitude'}
)
df_stations = df_stations.drop_duplicates(subset='station_id')
df_stations = df_stations.set_index('station_id')

print("Unique stations DataFrame created from start stations only:")
print(df_stations.head())

Unique stations DataFrame created from start stations only:
             latitude  longitude
station_id                      
3160.0      39.956619 -75.198624
3006.0      39.952202  -75.20311
3010.0      39.947109 -75.166183
3026.0       39.94138 -75.145638
3037.0      39.954239 -75.161377


In [ ]:
# Filter out stations with missing lat/long, like Station ID 3000
df_stations.dropna(subset=['latitude', 'longitude'], inplace=True)

# Prepare Coordinates in radians
station_coords_radians = np.radians(df_stations[['latitude', 'longitude']].values.astype(float))

# Calculate Haversine Distance Matrix
dist_matrix_radians = haversine_distances(station_coords_radians)

# Convert to Kilometers
earth_radius_km = 6371
dist_matrix_km = dist_matrix_radians * earth_radius_km

# Convert to a Long-Format DataFrame
df_distances = pd.DataFrame(dist_matrix_km, index=df_stations.index, columns=df_stations.index)
df_distances = df_distances.unstack().reset_index(name='distance_km')
df_distances.columns = ['origin_id', 'destination_id', 'distance_km']

print("Distances DataFrame (in km) created successfully:")
print(df_distances.head())

Distances DataFrame (in km) created successfully:
   origin_id  destination_id  distance_km
0     3160.0          3160.0     0.000000
1     3160.0          3006.0     0.622444
2     3160.0          3010.0     2.960575
3     3160.0          3026.0     4.824125
4     3160.0          3037.0     3.185790
